In [ ]:
# To add a new cell, type '# %%'
# To add a new markdown cell, type '# %% [markdown]'
# %%
# To add a new cell, type '# %%'
# To add a new markdown cell, type '# %% [markdown]'
# %%
#-------------------------------
#Funktionen:
#-------------------------------

# Print iterations progress
def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█', printEnd = "\r"):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
        printEnd    - Optional  : end character (e.g. "\r", "\r\n") (Str)
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print('\r%s |%s| %s%% %s' % (prefix, bar, percent, suffix), end = printEnd)
    # Print New Line on Complete
    if iteration == total: 
        print()


In [ ]:
import numpy as np
from osgeo import gdal
from osgeo import osr
from scipy import interpolate


#-------------------------------
#File einlesen:
#-------------------------------
fobj = open('Daten/Waldpunktwolke_1000.txt', "r") #Input File
InputSize = "1000"

pointlist = []

next(fobj)
for line in fobj:
    line = line.strip()
    line= line.split("\t")   
    pointlist.append(line)
fobj.close


In [ ]:
#-------------------------------
#Daten einlesen und nach Typ Filtern:
#-------------------------------

pointarray = np.array(pointlist) 
pointarray = pointarray.astype(float)  
print(len(pointarray))

#Neuer Array ohne Gebauede und Boden, Klassen 3, 4, 5
vegetationslist = []
bodenlist=[]

print("Vegetations-und Bodenliste einlesen:")
printProgressBar(0, len(pointarray), prefix = 'Progress:', suffix = 'Complete', length = 50)
j=0
for i in pointarray:
    printProgressBar(j+1, len(pointarray), prefix = 'Progress:', suffix = 'Complete', length = 50)
    j+=1
    if i[6] == 3 or i[6] ==4 or i[6] ==5:
        vegetationslist.append(i)
    if i[6] == 2:
        bodenlist.append(i)

vegarray = np.array(vegetationslist)        
bodenarray = np.array(bodenlist)




In [ ]:
#----------------------------------
#Extend Array with random numbers:
#----------------------------------
#max values
maxvals = np.amax(pointarray,axis=0) 
xmax=maxvals[0]
ymax=maxvals[1]
# print ("xmax:",xmax)
# print ("ymax:",ymax)

#min values
minvals = np.amin(pointarray,axis=0) 
xmin=minvals[0]
ymin=minvals[1]
# print ("xmin:",xmin)
# print ("ymin:",ymin)

#empty array with extent dimensions
nrows= int(np.ceil(ymax-ymin))
ncols= int(np.ceil(xmax-xmin))
z=2

# Erstellen der verschiedenen Arrays

bodencountout_array = np.empty((nrows,ncols))
bodencountout_array.fill(0)

bodenhoehencountout_array = np.empty((nrows,ncols))
bodenhoehencountout_array[:] = np.NaN

bodenhoehen_array = np.empty((nrows,ncols))
bodenhoehen_array[:] = np.NaN

vegcount_array = np.empty((nrows,ncols))
vegcount_array.fill(0)

vegcount_higher2_array = np.empty((nrows,ncols))
vegcount_higher2_array.fill(0)

veghohen_arry= np.empty((nrows,ncols))
veghohen_arry[:] = np.NaN

veghigher2_hoehen_array  = np.empty((nrows,ncols))
veghigher2_hoehen_array[:] = np.NaN

vegpoints_array = np.empty((nrows,ncols))
vegpoints_array[:] = np.NaN

bodenpoints_array = np.empty((nrows,ncols))
bodenpoints_array[:] = np.NaN

veghigher2_count_array = np.empty((nrows,ncols))
veghigher2_count_array[:] = np.NaN

In [ ]:
#----------------------------------
#Bodenpunkte zaehlen:
#----------------------------------
#Cellsize = float(raw_input("Bitte geben Sie die gewuenschte Rasterzellengroesse an:")) #Raw Input funktioniert in VisualStudio nicht
Cellsize = 1

print("Bodenpunkte in Rasterzellen aufteilen und Mittelwert pro Zelle berechnen:")
printProgressBar(0, len(bodenarray), prefix = 'Progress:', suffix = 'Complete', length = 50)
j=0
for i in bodenarray:
    printProgressBar(j+1, len(bodenarray), prefix = 'Progress:', suffix = 'Complete', length = 50)
    j+=1
    x=i[0]
    y=i[1]
    height = i[2]
    gx = int((x -xmin)/Cellsize)
    gy = int((y -ymax)/-Cellsize)  
    bodencountout_array[gy][gx] +=1
    bodenhoehencountout_array[gy][gx] +=1
    if np.isnan(bodenhoehencountout_array[gy][gx]) == True:
        bodenhoehen_array[gy][gx] = i[2]
    else:
        bodenhoehen_array[gy][gx] = np.mean([bodenhoehen_array[gy][gx],i[2]])

if bodencountout_array[gy][gx] == 0:
    bodencountout_array[gy][gx] +=1


##### Boden interpolation
##Dokumention : "https://modelhelptokyo.wordpress.com/2017/10/25/how-to-interpolate-missing-values-2d-python/"
a = np.arange(0, ncols)
b = np.arange(0, nrows)
#mask invalid values
array = np.ma.masked_invalid(bodenhoehen_array)
xx, yy = np.meshgrid(a, b)
#get only the valid values
x1 = xx[~array.mask]
y1 = yy[~array.mask]
newarr = array[~array.mask]

GD1 = interpolate.griddata((x1, y1), newarr.ravel(),
                          (xx, yy),
                             method='cubic')

print (GD1)
# %%
#----------------------------------
#Mittlere Hoehe der VegPunkte:
#----------------------------------
print("Vegetationspunkte in Rasterzellen aufteilen und erstellen eines Array mit Punkten uber 2 Meter:")
printProgressBar(0, len(vegarray), prefix = 'Progress:', suffix = 'Complete', length = 50)
j=0
for i in vegarray:
    printProgressBar(j+1, len(vegarray), prefix = 'Progress:', suffix = 'Complete', length = 50)
    j+=1
    x=i[0]
    y=i[1]
    height = i[2]
    #print(height-GD1[gy][gx])
    gx = int((x -xmin)/Cellsize)
    gy = int((y -ymax)/-Cellsize)
    vegcount_array[gy,gx]+=1
    veghohen_arry[gy,gx] = (i[2]-GD1[gy][gx])
    if height-GD1[gy][gx] >= 2:
        vegcount_higher2_array[gy,gx]+=1
        veghigher2_count_array[gy][gx]+=1
        if np.isnan(veghigher2_count_array[gy][gx]) == True:
            veghigher2_hoehen_array[gy][gx] = (i[2]-GD1[gy][gx])
        else:
            veghigher2_hoehen_array[gy][gx] = np.mean([veghigher2_hoehen_array[gy][gx],(i[2]-GD1[gy][gx])])



In [ ]:
#-------------------------------
#Index anzahl veg zu bodenpunkte veg/(veg-boden):
#-------------------------------
np.seterr(divide='ignore', invalid='ignore')
indexarray1 = np.empty((nrows,ncols))
indexarray1 = (vegcount_array/(vegcount_array+bodencountout_array))



In [ ]:
#-------------------------------
#Index anzahl veg ueber 2 m zu bodenpunkte veg/(veg-boden):
#-------------------------------
np.seterr(divide='ignore', invalid='ignore')
indexhoehen2m = np.empty((nrows,ncols))
indexhoehen2m = (vegcount_higher2_array/(vegcount_higher2_array+bodencountout_array))


In [ ]:
#-------------------------------
#OutRaster schreiben:
#-------------------------------


#Boden Raster
driver = gdal.GetDriverByName("GTiff")
dataset = driver.Create("Export/BodenCount_%s.tif"  % (InputSize), ncols, nrows, 1, gdal.GDT_Float32)
dataset.SetGeoTransform((xmin,1,0,ymax,0,-1))

#Koordinatensystem definieren:
dstSRS = osr.SpatialReference()
dstSRS.ImportFromEPSG(32632)
dest_wkt = dstSRS.ExportToWkt()

dataset.SetProjection(dest_wkt)

#Raster ausgeben:
bandout = dataset.GetRasterBand(1).WriteArray(bodencountout_array)
dataset.FlushCache()


In [ ]:
# #Bodenhoehen Raster
driver = gdal.GetDriverByName("GTiff")
dataset = driver.Create("Export/BodenHoehen_%s.tif" % (InputSize), ncols, nrows, 1, gdal.GDT_Float32)
dataset.SetGeoTransform((xmin,1,0,ymax,0,-1))

# #Koordinatensystem definieren:
dstSRS = osr.SpatialReference()
dstSRS.ImportFromEPSG(32632)
dest_wkt = dstSRS.ExportToWkt()

dataset.SetProjection(dest_wkt)

# #Raster ausgeben:
bandout = dataset.GetRasterBand(1).WriteArray(bodenhoehen_array)
dataset.FlushCache()

#%%
# #Bodenhoehen Raster
driver = gdal.GetDriverByName("GTiff")
dataset = driver.Create("Export/BodenInterpolation_%s.tif" % (InputSize), ncols, nrows, 1, gdal.GDT_Float32)
dataset.SetGeoTransform((xmin,1,0,ymax,0,-1))

# #Koordinatensystem definieren:
dstSRS = osr.SpatialReference()
dstSRS.ImportFromEPSG(32632)
dest_wkt = dstSRS.ExportToWkt()

dataset.SetProjection(dest_wkt)

# #Raster ausgeben:
bandout = dataset.GetRasterBand(1).WriteArray(GD1)
dataset.FlushCache()

In [ ]:

#Vegetation Zaehler Raster
driver = gdal.GetDriverByName("GTiff")
dataset = driver.Create("Export/VegCount_%s.tif" % (InputSize), ncols, nrows, 1, gdal.GDT_Float32)
dataset.SetGeoTransform((xmin,1,0,ymax,0,-1))

#Koordinatensystem definieren:
dstSRS = osr.SpatialReference()
dstSRS.ImportFromEPSG(32632)
dest_wkt = dstSRS.ExportToWkt()

dataset.SetProjection(dest_wkt)

#Raster ausgeben:
bandout = dataset.GetRasterBand(1).WriteArray(vegcount_array)
dataset.FlushCache()


In [ ]:

# #Vegetation Mittlere Hoehe Raster
driver = gdal.GetDriverByName("GTiff")
dataset = driver.Create("Export/VegHoehen_%s.tif" % (InputSize), ncols, nrows, 1, gdal.GDT_Float32)
dataset.SetGeoTransform((xmin,1,0,ymax,0,-1))

# #Koordinatensystem definieren:
dstSRS = osr.SpatialReference()
dstSRS.ImportFromEPSG(32632)
dest_wkt = dstSRS.ExportToWkt()

dataset.SetProjection(dest_wkt)

# #Raster ausgeben:
bandout = dataset.GetRasterBand(1).WriteArray(veghohen_arry)
dataset.FlushCache()

print ("done1")
# %%

# #Vegetation Standardabweichung Hoehe Raster
# driver = gdal.GetDriverByName("GTiff")
# dataset = driver.Create("Export/VegStd_1000.tif", ncols, nrows, 1, gdal.GDT_Float32)
# dataset.SetGeoTransform((xmin,1,0,ymax,0,-1))

# #Koordinatensystem definieren:
# dstSRS = osr.SpatialReference()
# dstSRS.ImportFromEPSG(32632)
# dest_wkt = dstSRS.ExportToWkt()

# dataset.SetProjection(dest_wkt)

# #Raster ausgeben:
# bandout = dataset.GetRasterBand(1).WriteArray(vegstdout_array)
# dataset.FlushCache()

In [ ]:
#Index Verhaeltnis VegPunkte zu Bodenpunkte Raster
driver = gdal.GetDriverByName("GTiff")
dataset = driver.Create("Export/Index1_%s.tif" % (InputSize), ncols, nrows, 1, gdal.GDT_Float32)
dataset.SetGeoTransform((xmin,1,0,ymax,0,-1))

#Koordinatensystem definieren:
dstSRS = osr.SpatialReference()
dstSRS.ImportFromEPSG(32632)
dest_wkt = dstSRS.ExportToWkt()

dataset.SetProjection(dest_wkt)

#Raster ausgeben:
bandout = dataset.GetRasterBand(1).WriteArray(indexarray1)
dataset.FlushCache()


In [ ]:
#Index Verhaeltnis VegPunkte zu Bodenpunkte Raster
driver = gdal.GetDriverByName("GTiff")
dataset = driver.Create("Export/Index2_higher2_%s.tif" % (InputSize), ncols, nrows, 1, gdal.GDT_Float32)
dataset.SetGeoTransform((xmin,1,0,ymax,0,-1))

#Koordinatensystem definieren:
dstSRS = osr.SpatialReference()
dstSRS.ImportFromEPSG(32632)
dest_wkt = dstSRS.ExportToWkt()

dataset.SetProjection(dest_wkt)

#Raster ausgeben:
bandout = dataset.GetRasterBand(1).WriteArray(indexhoehen2m)
dataset.FlushCache()

# # %%
# #Index Verhaeltnis VegPunkte zu Bodenpunkte Raster Saga
# driver = gdal.GetDriverByName("AAIGrid")
# dataset = driver.Create("Export/Index2_higher2_%s.tif" % (InputSize), ncols, nrows, 1, gdal.GDT_Float32)
# #dataset.SetGeoTransform((xmin,1,0,ymax,0,-1))

# #Koordinatensystem definieren:
# dstSRS = osr.SpatialReference()
# dstSRS.ImportFromEPSG(32632)
# dest_wkt = dstSRS.ExportToWkt()

# dataset.SetProjection(dest_wkt)

# #Raster ausgeben:
# bandout = dataset.GetRasterBand(1).WriteArray(vegstdout_array)
# dataset.FlushCache()

print ("finalDone")
